# Basic Azure Resource Management

## PowerShell

PowerShell is a powerful scripting and automation framework developed by Microsoft. It is designed for task automation and configuration management and is particularly useful for managing and automating Microsoft Windows environments. PowerShell uses a command-line interface with a scriptable approach, and it's built on the .NET Framework.

* [What is PowerShell?](https://learn.microsoft.com/powershell/scripting/overview?view=powershell-7.3&WT.mc_id=AZ-MVP-5004796)

## PowerShell and Microsoft Azure

When it comes to Microsoft Azure, PowerShell provides a robust set of cmdlets (pronounced "command-lets") that enable you to interact with and manage Azure resources, making it a valuable tool for working with Azure services. 

When you run a PowerShell cmdlet to, for example, create a virtual machine or retrieve information about an Azure resource, the cmdlet translates your request into an HTTP request to the relevant Azure REST API endpoint.

* [Azure PowerShell Documentation](https://learn.microsoft.com/powershell/azure/?view=azps-10.4.1&WT.mc_id=AZ-MVP-5004796)

## Resource Creation

In [2]:
# Import Module
Import-Module Az -Verbose

VERBOSE: Loading module from path '/root/.local/share/powershell/Modules/Az/10.4.1/Az.psd1'.


In [ ]:
#Create Azure Resource Group
New-AzResourceGroup -Name "MyResourceGroup" -Location "West US"

In [46]:
# Get Regions

Get-AzLocation | Select-Object -First 1
Get-AzLocation | Select-Object DisplayName, Location, PhysicalLocation, GeographyGroup | Format-Table



Location         : eastus
DisplayName      : East US
Type             : Region
Longitude        : -79.8164
Latitude         : 37.3719
PhysicalLocation : Virginia
RegionType       : Physical
RegionCategory   : Recommended
GeographyGroup   : US
PairedRegion     : [
                     {
                       "Name": "westus",
                       "Id": 
                   "/subscriptions/6bca53bc-98d9-4cd3-92e7-0364c7bffac4/locations/westus",
                       "SubscriptionId": null
                     }
                   ]
Providers        : {Microsoft.ContainerRegistry, Microsoft.AlertsManagement, microsoft.insights, 
                   Microsoft.DesktopVirtualization…}



DisplayName          Location           PhysicalLocation GeographyGroup
-----------          --------           ---------------- --------------
East US              eastus             Virginia         US
East US 2            eastus2            Virginia         US
South Central US     southcentralus     Te

In [19]:
#Create Azure Resource Group
$region = 'AustraliaEast'
New-AzResourceGroup -Name "MyResourceGroup$region" -Location $region


ResourceGroupName : MyResourceGroupAustraliaEast
Location          : australiaeast
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/6bca53bc-98d9-4cd3-92e7-0364c7bffac4/resourceGroups/MyResourceGr
                    oupAustraliaEast




In [29]:
# Create a storage account (attempt1)

$region = 'AustraliaEast'
$ResourceGroupName = "MyResourceGroup$region"
New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystorageaccount" -Location $region -SkuName Standard_LRS -AllowBlobPublicAccess $false

Default value of AllowBlobPublicAccess and AllowCrossTenantReplication settings on storage account will be changed to False in the future release. 
When AllowBlobPublicAccess is False on a storage account, container ACLs cannot be configured to allow anonymous access to blobs within the storage account. 
When AllowCrossTenantReplication is False on a storage account, cross AAD tenant object replication is not allowed when setting up Object Replication policies.
- The change is expected to take effect in Az version : '11.0.0'
- The change is expected to take effect in Az.Storage version : '6.0.0'
Cmdlet invocation changes :
    Old Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to True by defult.
    New Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to False by default.
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.
New-AzStorageA

Error: Command failed: SubmitCode: # Create a storage account (attempt1) ...

In [44]:
# Create a storage account (attempt2)
$uniqueId = [guid]::NewGuid().ToString().Substring(0,4)
$region = 'AustraliaEast'
$ResourceGroupName = "MyResourceGroup$region"
New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -Location $region -SkuName Standard_LRS -AllowBlobPublicAccess $false -verbose

Default value of AllowBlobPublicAccess and AllowCrossTenantReplication settings on storage account will be changed to False in the future release. 
When AllowBlobPublicAccess is False on a storage account, container ACLs cannot be configured to allow anonymous access to blobs within the storage account. 
When AllowCrossTenantReplication is False on a storage account, cross AAD tenant object replication is not allowed when setting up Object Replication policies.
- The change is expected to take effect in Az version : '11.0.0'
- The change is expected to take effect in Az.Storage version : '6.0.0'
Cmdlet invocation changes :
    Old Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to True by defult.
    New Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to False by default.
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.

StorageAccoun

In [80]:
#Remove Azure Storage account
$region = 'AustraliaEast'
$ResourceGroupName = $ResourceGroupName
Remove-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -Force -verbose
Get-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -verbose

VERBOSE: Performing the operation "Remove Storage Account" on target "mystgacce3b9".
Get-AzStorageAccount: 
Line |
   5 |  Get-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mys …
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The Resource 'Microsoft.Storage/storageAccounts/mystgacce3b9' under resource group 'network-prod-rg' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix


Error: Command failed: SubmitCode: #Remove Azure Storage account ...

In [81]:
# Create an Azure Virtual Network
$region = 'AustraliaEast'
$ResourceGroupName = 'network-prod-rg'
$VNetname = 'vnet-prod'
$subnetname = 'infraservers'
$subnetAddressPrefix = '10.0.0.0/24'

# Create a resource group
$ResourceGroup = Get-AzResourceGroup -Name $ResourceGroupName -ErrorAction SilentlyContinue

if ($null -eq $ResourceGroup)
{
    Write-Host "Creating Resource Group $ResourceGroupName in $region" -ForegroundColor Yellow
    $ResourceGroup = New-AzResourceGroup -Name $ResourceGroupName -Location $region -Force
}
else
{
    Write-Host "Resource Group $ResourceGroupName already exists in $region" -ForegroundColor Green
}

# Create a virtual network
$AzVNET = New-AzVirtualNetwork -ResourceGroupName $ResourceGroupName -Name $VNetname -AddressPrefix '10.0.0.0/16' -Location $region

# Create a subnet
$subnetConfig = Add-AzVirtualNetworkSubnetConfig -Name $subnetname -AddressPrefix $subnetAddressPrefix -VirtualNetwork $AzVNET



Resource Group network-prod-rg already exists in AustraliaEast

Confirm
Are you sure you want to overwrite resource 'vnet-prod'
[Y] Yes  [N] No  [S] Suspend  [?] Help(default is 'Y')

Error: Microsoft.DotNet.Interactive.NoSuitableKernelException: No kernel found for Microsoft.DotNet.Interactive.Commands.RequestHoverText with target kernel 'pwsh'.
   at Microsoft.DotNet.Interactive.Commands.KernelCommand.InvokeAsync(KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Commands\KernelCommand.cs:line 184
   at Microsoft.DotNet.Interactive.Kernel.HandleAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.cs:line 324
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.<BuildPipeline>b__6_0(KernelCommand command, KernelInvocationContext context, KernelPipelineContinuation _) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 60
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.SendAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 51

Error: Microsoft.DotNet.Interactive.NoSuitableKernelException: No kernel found for Microsoft.DotNet.Interactive.Commands.RequestHoverText with target kernel 'pwsh'.
   at Microsoft.DotNet.Interactive.Commands.KernelCommand.InvokeAsync(KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Commands\KernelCommand.cs:line 184
   at Microsoft.DotNet.Interactive.Kernel.HandleAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.cs:line 324
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.<BuildPipeline>b__6_0(KernelCommand command, KernelInvocationContext context, KernelPipelineContinuation _) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 60
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.SendAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 51

Error: Microsoft.DotNet.Interactive.NoSuitableKernelException: No kernel found for Microsoft.DotNet.Interactive.Commands.RequestHoverText with target kernel 'pwsh'.
   at Microsoft.DotNet.Interactive.Commands.KernelCommand.InvokeAsync(KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Commands\KernelCommand.cs:line 184
   at Microsoft.DotNet.Interactive.Kernel.HandleAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.cs:line 324
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.<BuildPipeline>b__6_0(KernelCommand command, KernelInvocationContext context, KernelPipelineContinuation _) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 60
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.SendAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 51

Error: Input request cancelled

Error: Input request cancelled

New-AzVirtualNetwork: 
Line |
  22 |  $AzVNET = New-AzVirtualNetwork -ResourceGroupName $ResourceGroupName  …
     |            ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Input request cancelled
Add-AzVirtualNetworkSubnetConfig: 
Line |
  25 |  … ubnetname -AddressPrefix $subnetAddressPrefix -VirtualNetwork $AzVNET
     |                                                                  ~~~~~~~
     | Cannot bind argument to parameter 'VirtualNetwork' because it is null.


Error: Command failed: SubmitCode: # Create an Azure Virtual Network ...

In [67]:
# Get full object output
# Alias (This is a pipeline to the Format-List cmdlet (fl is an alias for Format-List). It formats the output as a list of properties for each object. This can make it easier to read the details of the virtual network.)

Get-AzVirtualNetwork -ResourceGroupName $ResourceGroupName -Name $VNetname # | fl


ResourceGroupName Name      Location      ProvisioningState EnableDdosProtection
----------------- ----      --------      ----------------- --------------------
network-prod-rg   vnet-prod australiaeast Succeeded         False



In [79]:
# Alias

Get-Alias  | Select-Object -First 2


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Alias           ? -> Where-Object                                             
Alias           % -> ForEach-Object                                           



In [ ]:
#splat

$configData = @{
    ResourceGroupName = "MyResourceGroup"
    Location = "West US"
    StorageAccountName = "stgacctest100"
}

try {
    New-AzStorageAccount -ResourceGroupName $configData.ResourceGroupName -Name $configData.StorageAccountName -Location $configData.Location -SkuName Standard_LRS
} catch {
    Write-Error "Failed to create storage account: $_"
}

In [91]:
#splat

$configData = @{
    "ResourceGroupName" = "MyResourceGroup"
    "Location" = "West US"
    "StorageAccountName" = "stgacctest100"
    "SkuName" = "Standard_LRS"

}

try {
    New-AzStorageAccount @configData
} catch {
    Write-Error "Failed to create storage account: $_"
}

Default value of AllowBlobPublicAccess and AllowCrossTenantReplication settings on storage account will be changed to False in the future release. 
When AllowBlobPublicAccess is False on a storage account, container ACLs cannot be configured to allow anonymous access to blobs within the storage account. 
When AllowCrossTenantReplication is False on a storage account, cross AAD tenant object replication is not allowed when setting up Object Replication policies.
- The change is expected to take effect in Az version : '11.0.0'
- The change is expected to take effect in Az.Storage version : '6.0.0'
Cmdlet invocation changes :
    Old Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to True by defult.
    New Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to False by default.
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.
New-AzStorageA

Error: Command failed: SubmitCode: #splat ...